<a href="https://colab.research.google.com/github/cacheop/facemask-detection/blob/master/Mask_prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

### Mount Google Drive

In [0]:
from datetime import datetime

now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y--%H-%M-%S")
print("date and time:",date_time)	

date and time: 06-06-2020--15-37-51


In [0]:
import tensorflow as tf
import os
from psutil import virtual_memory

print("Current Tensorflow version: ", tf.__version__)


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

!pip freeze | grep tensor


Current Tensorflow version:  2.2.0
Found GPU at: /device:GPU:0
Tue Jun  2 15:11:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    34W / 250W |    353MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:             

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!ln -s "/content/drive/My Drive/Colab Notebooks/medical-masks-dataset/" "/content/medical-masks-dataset"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -q pillow lxml Cython jupyter matplotlib pandas opencv-python contextlib2 lxml xmltodict pycocotools

In [0]:
%cd /content/medical-masks-dataset
!git clone https://github.com/tensorflow/models/

!mkdir training
!mkdir inference_graph
!mkdir -p images/train
!mkdir -p images/test
%cd models/research/object_detection


/content/drive/My Drive/Colab Notebooks/medical-masks-dataset
Cloning into 'models'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 34420 (delta 2), reused 8 (delta 0), pack-reused 34407
Receiving objects: 100% (34420/34420), 512.36 MiB | 14.74 MiB/s, done.
Resolving deltas: 100% (22206/22206), done.
Checking out files: 100% (2484/2484), done.
mkdir: cannot create directory ‘inference_graph’: File exists
/content/drive/My Drive/Colab Notebooks/medical-masks-dataset/models/research/object_detection


In [0]:
import os
os.environ['PYTHONPATH'] = "{}/content/obj_detect_api/models:/content/obj_detect_api/models/research:/content/obj_detect_api/models/research/slim".format(os.environ['PYTHONPATH'])

%cd ..

!protoc ./object_detection/protos/*.proto --python_out=.

!protoc --version

/content/drive/My Drive/Colab Notebooks/medical-masks-dataset/models/research
libprotoc 3.0.0


In [0]:
import os
import cv2
import matplotlib.pyplot as plt
import xmltodict
import random
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm, trange

# Split data folder into Test and Train


In [0]:
from random import choice
import os
import shutil
from glob import glob                                                           
import cv2 

#setup dir names
HOME_DIR = "/content/medical-masks-dataset/data/"
img_path = HOME_DIR + 'source/images/' #dir with images 
xml_path = HOME_DIR + 'source/labels/' #dir with annotations 

train_path = HOME_DIR + 'train/'
train_path_images = train_path + 'images/'
train_path_xml = train_path + 'labels/'
train_path_csv = train_path + 'csv/'

test_path = HOME_DIR + 'test/'
test_path_images = test_path + 'images/'
test_path_xml = test_path + 'labels/'
test_path_csv = test_path + 'csv/'

In [0]:
pngs = glob(img_path +'*.png')
jpegs = glob(img_path +'*.jpeg')
JPGs = glob(img_path +'*.JPG')

print("Converting pngs...")
for j in pngs:
    img = cv2.imread(j)
    cv2.imwrite(j[:-3] + 'jpg', img)
    print(j)
    os.remove(j)

print("Renaming jpeg to jpg...")
for name in jpegs:
    new_name = name[:-5] +'.jpg'
    print(name, new_name)
    os.rename(name, new_name)


print("Renaming JPG to jpg...")
for name in JPGs:
    new_name = name[:-4] +'.jpg'
    print(name, new_name)
    os.rename(name, new_name)

# TO-DO: fix XML reference from PNG and JPEG to JPG

Converting pngs...
Renaming jpeg to jpg...
Renaming JPG to jpg...


In [0]:
#arrays to store file names
imgs = []
xmls = []

#split ratio 
train_ratio = 0.8
test_ratio = 0.1

#total count of imgs + annotations
total_img_count = len(os.listdir(img_path))/2
total_xml_count = len(os.listdir(xml_path))/2
print ('There are', total_img_count, 'images and ', total_xml_count, 'annotations.')

#they gotta be the same count
if (total_img_count != total_img_count):
    print ("WARNING: mismatch!")

There are 339.0 images and  339.0 annotations.


In [0]:
import os.path

not_paired = 0

#sorting files to corresponding arrays
for (dirname, dirs, files) in os.walk(xml_path):
    for filename in files:
        img_file_pair = img_path + filename[:-4] + '.jpg'
        if os.path.isfile(img_file_pair):
            xmls.append(filename)
        else:
            print("will remove", filename)
            not_paired += 1
            os.remove(dirname+filename)
print('Removed ', not_paired, 'not paired XML files for good')

for (dirname, dirs, files) in os.walk(img_path):
    for filename in files:
        imgs.append(filename)

total_imgs = len(imgs)
total_xmls = len(xmls)
print('Images', total_imgs)
print('XML files', total_xmls)

#counting range for cycles

count_for_train = int(len(imgs)*train_ratio)
count_for_test = int(len(imgs)*test_ratio)

print("Split files: ", count_for_train, count_for_test)

#cycle for train dir images
for x in range(count_for_train):
    file_jpg = choice(imgs) # get name of random image from origin dir
    file_xml = file_jpg[:-4] +'.xml' # get name of corresponding annotation file
    print("Train:    [", file_jpg,"]", "\n[", file_xml,"]\n\n")

    #move both files into train dir
    shutil.move(os.path.join(img_path, file_jpg), os.path.join(train_path_images, file_jpg))
    imgs.remove(file_jpg)

    shutil.move(os.path.join(xml_path, file_xml), os.path.join(train_path_xml, file_xml))
    xmls.remove(file_xml)

#cycle for test dir   
for x in range(count_for_test):

    file_jpg = choice(imgs) # get name of random image from origin dir
    file_xml = file_jpg[:-4] +'.xml' # get name of corresponding annotation file
    print("Test:  [", file_jpg,"]", "\n[", file_xml,"]\n\n")


    #move both files into test dir
    shutil.move(os.path.join(img_path, file_jpg), os.path.join(test_path_images, file_jpg))
    imgs.remove(file_jpg)

    shutil.move(os.path.join(xml_path, file_xml), os.path.join(test_path_xml, file_xml))
    xmls.remove(file_xml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)

#summary information after splitting
print('Total images: ', total_img_count)

Removed  0 not paired XML files for good
Images 678
XML files 678
Split files:  542 67
Train:    [ 1024x576_939430607723.jpg ] 
[ 1024x576_939430607723.xml ]


Train:    [ depositphotos_196902688-stock-photo-mother-daughter-protective-masks-hugging.jpg ] 
[ depositphotos_196902688-stock-photo-mother-daughter-protective-masks-hugging.xml ]


Train:    [ unnamed.jpg ] 
[ unnamed.xml ]


Train:    [ ST_20190917_SPTHAZE_5120715.jpg ] 
[ ST_20190917_SPTHAZE_5120715.xml ]


Train:    [ 20200127000706.jpg ] 
[ 20200127000706.xml ]


Train:    [ 0109-00176-096b1.jpg ] 
[ 0109-00176-096b1.xml ]


Train:    [ d81b382a5627d2aa15e4df08c67c6e4e_w.jpg ] 
[ d81b382a5627d2aa15e4df08c67c6e4e_w.xml ]


Train:    [ Coronavirus-1024x683.jpg ] 
[ Coronavirus-1024x683.xml ]


Train:    [ photo (2).jpg ] 
[ photo (2).xml ]


Train:    [ S1-EV360_FACEMA_M_20200130144349.jpg ] 
[ S1-EV360_FACEMA_M_20200130144349.xml ]


Train:    [ stsciRq.jpg ] 
[ stsciRq.xml ]


Train:    [ 2020013023283369938.jpg ] 
[ 20200

In [0]:
print('Images in train dir:', len(os.listdir(train_path_images)))
print('Images in test dir:', len(os.listdir(test_path_images)))
print('XMLs in train dir:', len(os.listdir(train_path_xml)))
print('XMLs in test dir:', len(os.listdir(test_path_xml)))

Images in train dir: 542
Images in test dir: 67
XMLs in train dir: 542
XMLs in test dir: 67


# XML to CSV

In [0]:
from glob import glob                                                           
from tqdm import tqdm, trange
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

def xml_to_csv(path):
    print("------------->", path)
    xml_list = []
    for xml_file in tqdm(glob(path + '/*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [0]:
print('Converting XMLs in folder: ', train_path_xml, '...\n')
train_xml_df = xml_to_csv(train_path_xml)
train_xml_df.head()


  0%|          | 0/542 [00:00<?, ?it/s]

Converting XMLs in folder:  /content/medical-masks-dataset/data/train/labels/ ...

-------------> /content/medical-masks-dataset/data/train/labels/


100%|██████████| 542/542 [04:12<00:00,  2.14it/s]


,filename,width,height,class,xmin,ymin,xmax,ymax
0,-1x-1.jpg,2000,1363,good,10,750,252,1164
1,-1x-1.jpg,2000,1363,bad,805,43,886,133
2,-1x-1.jpg,2000,1363,good,584,34,700,129
3,-1x-1.jpg,2000,1363,good,52,1,160,56
4,-1x-1.jpg,2000,1363,good,461,181,569,292


In [0]:
print('Converting XMLs in folder: ', test_path_xml, '...\n')
test_xml_df = xml_to_csv(test_path_xml)
test_xml_df.head()

  0%|          | 0/67 [00:00<?, ?it/s]

Converting XMLs in folder:  /content/medical-masks-dataset/data/test/labels/ ...

-------------> /content/medical-masks-dataset/data/test/labels/


100%|██████████| 67/67 [00:31<00:00,  2.11it/s]


,filename,width,height,class,xmin,ymin,xmax,ymax
0,000_1OC3DT.jpg,1024,682,good,22,124,94,183
1,000_1OC3DT.jpg,1024,682,good,161,108,228,176
2,000_1OC3DT.jpg,1024,682,good,287,105,357,173
3,000_1OC3DT.jpg,1024,682,good,384,140,447,187
4,000_1OC3DT.jpg,1024,682,good,436,133,500,200


In [0]:
train_xml_df.to_csv(train_path_csv + 'train_labels.csv', index=None)
test_xml_df.to_csv(test_path_csv + 'test_labels.csv', index=None)

print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [0]:
!pip install tensorflow-object-detection-api

In [0]:
from google.protobuf import text_format
from tensorflow.python.platform import gfile
import io
import pandas as pd
from object_detection.utils import dataset_util


from PIL import Image
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'good':
        return 1
    elif row_label =='bad':
        return 2
    elif row_label =='none': 
        return 3
    return 0

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    """print('group', group)
    print('path', path)
    print('width=', width, 'height=',height)"""

    filename = group.filename.encode('utf8')
    #print('filename', filename)

    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [0]:
train_examples = pd.read_csv(train_path_csv + 'train_labels.csv')
test_examples = pd.read_csv(test_path_csv + 'test_labels.csv')
print(len(train_examples))
print(len(test_examples))

3039
490


In [0]:
import tensorflow as tf
from tqdm import tqdm, trange

def generate_tfrecord(csv_file, record_output_path, record_name, path_to_images):
    writer = tf.io.TFRecordWriter(record_output_path + record_name)

    examples = pd.read_csv(csv_file)
    grouped = split(examples, 'filename')
    for group in tqdm(grouped):
        tf_example = create_tf_example(group, path_to_images)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('\nSuccessfully created ', record_name, ': {}'.format(record_output_path))

    return

In [0]:
train_csv_file = train_path_csv + 'train_labels.csv'
generate_tfrecord(train_csv_file, HOME_DIR, 'train.record', train_path_images)

test_csv_file = test_path_csv + 'test_labels.csv'
generate_tfrecord(test_csv_file, HOME_DIR, 'test.record', test_path_images)